In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

#@title import matplotlib and set defaults
from matplotlib import rcParams 
from matplotlib import pyplot as plt
rcParams['figure.figsize'] = [20, 4]
rcParams['font.size'] =15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

In [2]:
#@title groupings of brain regions
regions = ["vis ctx", "thal", "hipp", "other ctx", "midbrain",  "basal ganglia", "subplate"]
brain_groups = [["VISa", "VISam", "VISl", "VISp", "VISpm", "VISrl"], # visual cortex
                ["CL", "LD", "LGd", "LH", "LP", "MD", "MG", "PO", "POL", "PT", "RT", "SPF", "TH", "VAL", "VPL", "VPM"], # thalamus
                ["CA", "CA1", "CA2", "CA3", "DG", "SUB", "POST"], # hippocampal
                ["ACA", "AUD", "COA", "DP", "ILA", "MOp", "MOs", "OLF", "ORB", "ORBm", "PIR", "PL", "SSp", "SSs", "RSP"," TT"], # non-visual cortex
                ["APN", "IC", "MB", "MRN", "NB", "PAG", "RN", "SCs", "SCm", "SCig", "SCsg", "ZI"], # midbrain
                ["ACB", "CP", "GPe", "LS", "LSc", "LSr", "MS", "OT", "SNr", "SI"], # basal ganglia 
                ["BLA", "BMA", "EP", "EPd", "MEA"] # cortical subplate
               ]

In [ ]:
import glob, os, sys
import numpy as np
from scipy.stats import zscore, spearmanr, mannwhitneyu, pearsonr
import time
import imp
import steinmetz_loader
from scipy.sparse import csr_matrix

imp.reload(steinmetz_loader)

dataroot = 'G:/steinmetz-data/Spikes'
fdir = glob.glob(os.path.join(dataroot, "*", ""))

dt = 1/100
dT = 2.5
T0 = .5

dat = []

for idir in range(1):
    # good cells and brain regions
    good_cells, brain_region, br = steinmetz_loader.get_good_cells(fdir[idir])
    
    # event types
    response, vis_right, vis_left, feedback_type = steinmetz_loader.get_event_types(fdir[idir])
    
    # event timing
    response_times, visual_times, rsp, gocue, feedback_time = steinmetz_loader.get_event_times(fdir[idir])    

    # get passive trials
    vis_times_p, vis_right_p, vis_left_p = steinmetz_loader.get_passive(fdir[idir])
    visual_times = np.vstack((visual_times, vis_times_p))
    vis_right = np.hstack((vis_right, vis_right_p))
    vis_left  = np.hstack((vis_left, vis_left_p))
    
    # wheel traces
    stimes, sclust    = steinmetz_loader.get_spikes(fdir[idir])
    
    # only care about spikes during trials
    wheel, wheel_times = steinmetz_loader.get_wheel(fdir[idir])
    
    # load the pupil
    pup, xy, pup_times = steinmetz_loader.get_pup(fdir[idir])

    # load the LFP
    L, ba_lfp = steinmetz_loader.get_LFP(fdir[idir], br, visual_times-T0, dT, dt, T0)
    
    # trials loader
    S  = steinmetz_loader.psth(stimes, sclust,   visual_times-T0, dT, dt)
    
    # wheel trials
    W = steinmetz_loader.wpsth(wheel, wheel_times,   visual_times-T0, dT, dt)
    
    # pupil loader
    P = steinmetz_loader.ppsth(pup, pup_times,   visual_times-T0, dT, dt)
    
    # add spike waveform information
    twav, w, u = steinmetz_loader.get_waves(fdir[idir])
    
    
    good_cells = good_cells * (np.mean(S, axis=(1,2))>0)
    S  = S[good_cells].astype('int8') 
   
    dat.append({})
    ntrials = len(dat[idir]['response'])
    
    dat[idir]['brain_area'] = brain_region[good_cells]
    dat[idir]['spks'] = S[:, :ntrials, :]
    dat[idir]['wheel'] = W[np.newaxis, :ntrials, :]
    dat[idir]['pupil'] = P[:, :ntrials, :]
    dat[idir]['response'] = response
    dat[idir]['contrast_right'] = vis_right[:ntrials]
    dat[idir]['contrast_left'] = vis_left[:ntrials]
    dat[idir]['response_time'] = rsp
    dat[idir]['feedback_time'] = feedback_time
    dat[idir]['feedback_type'] = feedback_type  
    dat[idir]['gocue'] = gocue
    dat[idir]['mouse_name'] = fdir[idir].split('\\')[1].split('_')[0]
    dat[idir]['date_exp'] = fdir[idir].split('\\')[1].split('_')[1]
    dat[idir]['trough_to_peak'] = twav[good_cells].flatten()
    dat[idir]['waveform_w'] = w[good_cells].astype('float32')
    dat[idir]['waveform_u'] = u[good_cells].astype('float32')
    dat[idir]['bin_size'] = dt
    dat[idir]['stim_onset'] = T0
    
    dat[idir]['spks_passive'] = S[:, ntrials:, :]
    dat[idir]['wheel_passive'] = W[np.newaxis, ntrials:, :]
    dat[idir]['pupil_passive'] = P[:, ntrials:, :]
    dat[idir]['lfp_passive'] = L[:, ntrials:, :]
    dat[idir]['contrast_right_passive'] = vis_right[ntrials:]
    dat[idir]['contrast_left_passive'] = vis_left[ntrials:]
        
    # add LFP
    L, ba_lfp = steinmetz_loader.get_LFP(fdir[idir], br, visual_times-T0, dT, dt, T0)
    dat[idir]['lfp'] = L[:, :ntrials, :]
    dat[idir]['lfp_passive'] = L[:, ntrials:, :]
    dat[idir]['brain_area_lfp'] = ba_lfp
  
    #S  = np.reshape(S[good_cells], (np.sum(good_cells), -1))
    #sall.append(csr_matrix(S))
    

In [7]:
dat[0].keys()

dict_keys(['brain_area', 'spks', 'wheel', 'pupil', 'response', 'contrast_right', 'contrast_left', 'response_time', 'feedback_time', 'feedback_type', 'gocue', 'mouse_name', 'date_exp', 'trough_to_peak', 'waveform_w', 'waveform_u', 'bin_size', 'stim_onset', 'spks_passive', 'wheel_passive', 'pupil_passive', 'lfp_passive', 'contrast_right_passive', 'contrast_left_passive', 'lfp', 'brain_area_lfp'])

In [88]:
np.savez_compressed('G:/steinmetz-data/steinmetz_part0.npz', dat = dat[:13])
np.savez_compressed('G:/steinmetz-data/steinmetz_part1.npz', dat = dat[13:26])
np.savez_compressed('G:/steinmetz-data/steinmetz_part2.npz', dat = dat[26:])